# Create Dataset

 This script generates 8 different datasets for the following QA Tasks:
 * `asa`
 * `phase1_duration`
 * `hospital_duration`
 * `hospital_admission`
 * `icu_duration`
 * `icu_admission`
 * `unplanned_admit`
 * `hospital_mortality`

 These datasets include input data and labels.

In [1]:
import logging
from pathlib import Path

import pandas as pd
from llm_utils import DataPaths

from make_dataset.dataset import DatasetBuilder

logging.basicConfig(level=logging.INFO)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

paths = DataPaths(project_dir=Path(__file__).parent.parent.parent, data_version=7)

paths.register(name="adt_raw", path=paths.raw / "2023-05-11_ADT.feather")
paths.register(name="case_raw", path=paths.raw / "2023-05-11_Case.feather")
paths.register(name="preanes_notes_raw", path=paths.raw / "2023-05-02_PreAnesNotes.feather")
paths.register(name="last10_notes_raw", path=paths.raw / "2023-05-02_LastTenNotes.feather")

PosixPath('/home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/raw/2023-05-02_LastTenNotes.feather')

 ## Build All Datasets for QA Prediction Outcome + Data Split + Note Type Combinations

In [2]:
builder = DatasetBuilder(paths=paths, dataset_version="dataset4")

INFO:BuilderLogger:Loading EncounterEvents.


ADTEvents Pipeline: 0:load_data > 1:clean_data > 2:compute_adt_location > 3:drop_adt_cols > 4:compute_key_info_from_adt_events
Running steps: [2, 3]


  0%|          | 0/2 [00:00<?, ?it/s]

INFO:FileLogger:Loaded: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/interim/adt_location.feather
INFO:PipelineLogger:Executed step num: 2, name: compute_adt_location
INFO:PipelineLogger:Executed step num: 3, name: drop_adt_cols
INFO:FileLogger:Loaded: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/raw/2023-05-11_ADT.feather
INFO:PipelineLogger:Executed step num: 0, name: load_data


CaseEvents Pipeline: 0:load_data > 1:clean_data > 2:transform_data
Running steps: [2]


...

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:FileLogger:Loaded: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/interim/preanes_notes_collated.feather
INFO:PipelineLogger:Executed step num: 3, name: collate_notes
INFO:FileLogger:Loaded: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/raw/2023-05-02_PreAnesNotes.feather
INFO:PipelineLogger:Executed step num: 0, name: load_data
INFO:BuilderLogger:Loading Last 10 Clinical Notes


LastTenNote Pipeline: 0:load_data > 1:clean_data > 2:process_data > 3:collate_notes
Running steps: [3]


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:FileLogger:Loaded: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/interim/last10_notes_collated.feather
INFO:PipelineLogger:Executed step num: 3, name: collate_notes
INFO:FileLogger:Loaded: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/raw/2023-05-02_LastTenNotes.feather
INFO:PipelineLogger:Executed step num: 0, name: load_data
INFO:BuilderLogger:Starting Preanes Note Count: 129143
INFO:BuilderLogger:Preanes Note Count (remove total note text < 100 tokens): 129139
INFO:BuilderLogger:Starting Last10 Note Count: 48215
INFO:BuilderLogger:Last10 Note Count (remove total note text < 100 tokens): 47551
INFO:BuilderLogger:Common Note ProcID Count: 47494


In [3]:
# Encounter Events, Cases, Notes, ADT
E = builder.E
C = E.case
A = E.adt
PN = builder.preanes_notes
L10N = builder.last10_notes
common_note_ids = builder.common_note_proc_ids

# Raw Data Statistics
raw_cases = C.raw_df
print(f"Raw Case Count: {raw_cases.shape[0]}")
print(f"Case Date Range: {raw_cases.DOS.min()} - {raw_cases.DOS.max()}")
print("\n")
raw_adt = A.raw_df
print(f"Raw ADT Count: {raw_adt.shape[0]}")
print(f"ADT Date Range: {raw_adt.EVENT_TIME.min()} - {raw_adt.EVENT_TIME.max()}")
print("\n")
preanes_notes = PN.raw_df
print(f"Raw Preanes Notes Count: {preanes_notes.shape[0]}")
print(
    "Preanes Notes Date Range: "
    f"{preanes_notes.NoteServiceDate.min()} - {preanes_notes.NoteServiceDate.max()}"
)
print("\n")
last10_notes = L10N.raw_df
print(f"Raw Last 10 Notes Count: {last10_notes.shape[0]}")
print(
    "Last 10 Notes Date Range: "
    f"{last10_notes.NoteServiceDate.min()} - {last10_notes.NoteServiceDate.max()}"
)
print("\n")
# Get Final Cases with and without Durations prior to building individual datasets
cases_without_duration = E.case.df.copy()
cases_with_duration = E.case_durations.copy()
patients_without_duration = cases_without_duration.drop_duplicates("PAT_ID")
non_organ_donor_cases_without_duration = patients_without_duration.query(
    "IsOrganDonorCase == False"
)
patients_with_duration = cases_with_duration.drop_duplicates("PAT_ID")
non_organ_donor_cases_with_duration = patients_with_duration.query("IsOrganDonorCase == False")
print(f"Case without Duration Count: {cases_without_duration.shape[0]}")
print(
    f"Case without Duration (Patient:Case uniquification) Count: {len(patients_without_duration)}"  # noqa
)
print(
    "Case without Duration Non-Organ Donors: ",
    len(non_organ_donor_cases_without_duration),
)
print("\n")
print(f"Case with Duration Count: {cases_with_duration.shape[0]}")
print(f"Case with Duration (Patient:Case uniquification) Count: {len(patients_with_duration)}")
print("Case with Duration Non-Organ Donors: ", len(non_organ_donor_cases_with_duration))

Raw Case Count: 137535
Case Date Range: 2021-04-01 - 2023-05-05


Raw ADT Count: 2253286
ADT Date Range: 2021-01-12 04:37:00 - 2023-03-26 23:51:00


Raw Preanes Notes Count: 133500
Preanes Notes Date Range: 2021-03-17 08:33:00 - 2023-03-28 14:37:00


Raw Last 10 Notes Count: 226821
Last 10 Notes Date Range: 2004-05-17 09:30:00 - 2023-03-24 20:36:00


Case without Duration Count: 125881
Case without Duration (Patient:Case uniquification) Count: 85208
Case without Duration Non-Organ Donors:  85128


Case with Duration Count: 105815
Case with Duration (Patient:Case uniquification) Count: 77087
Case with Duration Non-Organ Donors:  76988


In [4]:
# Create Datasets and Save to disk
logging.info("Create ASA Dataset")
asa = builder.create_dataset(dataset_type="asa")

INFO:root:Create ASA Dataset
INFO:BuilderLogger:Starting Num Cases: 125881
INFO:BuilderLogger:Num Cases (filter by ProcIDs where we have PreAnes & Last10 Notes): 46472


Select up to 1 cases per Patient (Dispatch):   0%|          | 0/42320 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/42320 [00:00<?, ?it/s]

INFO:BuilderLogger:1 Case Per Patient. Num Cases: 42320
INFO:BuilderLogger:After Downsample. Num Cases: 1250
INFO:BuilderLogger:Preanes Inference Split: 1000
INFO:BuilderLogger:Preanes Fewshot Split: 250
INFO:BuilderLogger:Last10 Inference Split: 1000
INFO:BuilderLogger:Last10 Fewshot Split: 250
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/asa-preanes-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/asa-preanes-fewshot.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/asa-last10-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/asa-last10-fewshot.feather


In [5]:
logging.info("Create Phase 1 Duration Dataset")
phase1 = builder.create_dataset(dataset_type="phase1_duration")

INFO:root:Create Phase 1 Duration Dataset
INFO:BuilderLogger:Starting Num Cases: 105815
INFO:BuilderLogger:Num Cases (filter by ProcIDs where we have PreAnes & Last10 Notes): 39816
INFO:BuilderLogger:Remove Organ Donor Cases. Num Cases: 39804


Select up to 1 cases per Patient (Dispatch):   0%|          | 0/36693 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/36693 [00:00<?, ?it/s]

INFO:BuilderLogger:1 Case Per Patient. Num Cases: 36693
INFO:BuilderLogger:After Downsample. Num Cases: 1250
INFO:BuilderLogger:Preanes Inference Split: 1000
INFO:BuilderLogger:Preanes Fewshot Split: 250
INFO:BuilderLogger:Last10 Inference Split: 1000
INFO:BuilderLogger:Last10 Fewshot Split: 250
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/phase1_duration-preanes-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/phase1_duration-preanes-fewshot.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/phase1_duration-last10-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/phase1_duration-last10-fewshot.feather


In [6]:
logging.info("Create Hospital Duration Dataset")
hospital_duration = builder.create_dataset(dataset_type="hospital_duration")

INFO:root:Create Hospital Duration Dataset
INFO:BuilderLogger:Starting Num Cases: 105815
INFO:BuilderLogger:Num Cases (filter by ProcIDs where we have PreAnes & Last10 Notes): 39816
INFO:BuilderLogger:Remove Organ Donor Cases. Num Cases: 39804


Select up to 1 cases per Patient (Dispatch):   0%|          | 0/36693 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/36693 [00:00<?, ?it/s]

INFO:BuilderLogger:1 Case Per Patient. Num Cases: 36693
INFO:BuilderLogger:After Downsample. Num Cases: 1250
INFO:BuilderLogger:Preanes Inference Split: 1000
INFO:BuilderLogger:Preanes Fewshot Split: 250
INFO:BuilderLogger:Last10 Inference Split: 1000
INFO:BuilderLogger:Last10 Fewshot Split: 250
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_duration-preanes-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_duration-preanes-fewshot.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_duration-last10-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_duration-last10-fewshot.feather


In [7]:
logging.info("Create Hospital Admission Dataset")
hospital_admission = builder.create_dataset(dataset_type="hospital_admission")

INFO:root:Create Hospital Admission Dataset
INFO:BuilderLogger:Starting Num Cases: 105815
INFO:BuilderLogger:Num Cases (filter by ProcIDs where we have PreAnes & Last10 Notes): 39816
INFO:BuilderLogger:Remove Organ Donor Cases. Num Cases: 39804


Select up to 1 cases per Patient (Dispatch):   0%|          | 0/36693 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/36693 [00:00<?, ?it/s]

INFO:BuilderLogger:1 Case Per Patient. Num Cases: 36693
INFO:BuilderLogger:After Downsample. Num Cases: 1250
INFO:BuilderLogger:Preanes Inference Split: 1000
INFO:BuilderLogger:Preanes Fewshot Split: 250
INFO:BuilderLogger:Last10 Inference Split: 1000
INFO:BuilderLogger:Last10 Fewshot Split: 250
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_admission-preanes-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_admission-preanes-fewshot.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_admission-last10-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_admission-last10-fewshot.feather


In [8]:
logging.info("Create ICU Duration Dataset")
icu_duration = builder.create_dataset(dataset_type="icu_duration")

INFO:root:Create ICU Duration Dataset
INFO:BuilderLogger:Starting Num Cases: 105815
INFO:BuilderLogger:Num Cases (filter by ProcIDs where we have PreAnes & Last10 Notes): 39816
INFO:BuilderLogger:Remove Organ Donor Cases. Num Cases: 39804
INFO:BuilderLogger:ICU Num Cases: 677
INFO:BuilderLogger:Non-ICU Num Cases: 39127


Select up to 1 cases per Patient (Dispatch):   0%|          | 0/661 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/661 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Dispatch):   0%|          | 0/36032 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/36032 [00:00<?, ?it/s]

INFO:BuilderLogger:1 Case Per Patient. ICU Num Cases: 661
INFO:BuilderLogger:1 Case Per Patient. Non-ICU Num Cases: 36032
INFO:BuilderLogger:1 Case Per Patient. Non-ICU Num Cases: 36032
INFO:BuilderLogger:After Downsample. Num Cases: 1250
INFO:BuilderLogger:Preanes Inference Split: 1000
INFO:BuilderLogger:Preanes Fewshot Split: 250
INFO:BuilderLogger:Last10 Inference Split: 1000
INFO:BuilderLogger:Last10 Fewshot Split: 250
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/icu_duration-preanes-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/icu_duration-preanes-fewshot.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/icu_duration-last10-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/icu_duration-last10-fewshot

In [9]:
logging.info("Create ICU Admission Dataset")
icu_admission = builder.create_dataset(dataset_type="icu_admission")

INFO:root:Create ICU Admission Dataset
INFO:BuilderLogger:Starting Num Cases: 105815
INFO:BuilderLogger:Num Cases (filter by ProcIDs where we have PreAnes & Last10 Notes): 39816
INFO:BuilderLogger:Remove Organ Donor Cases. Num Cases: 39804
INFO:BuilderLogger:ICU Num Cases: 677
INFO:BuilderLogger:Non-ICU Num Cases: 39127


Select up to 1 cases per Patient (Dispatch):   0%|          | 0/661 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/661 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Dispatch):   0%|          | 0/36032 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/36032 [00:00<?, ?it/s]

INFO:BuilderLogger:1 Case Per Patient. ICU Num Cases: 661
INFO:BuilderLogger:1 Case Per Patient. Non-ICU Num Cases: 36032
INFO:BuilderLogger:1 Case Per Patient. Non-ICU Num Cases: 36032
INFO:BuilderLogger:After Downsample. Num Cases: 1250
INFO:BuilderLogger:Preanes Inference Split: 1000
INFO:BuilderLogger:Preanes Fewshot Split: 250
INFO:BuilderLogger:Last10 Inference Split: 1000
INFO:BuilderLogger:Last10 Fewshot Split: 250
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/icu_admission-preanes-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/icu_admission-preanes-fewshot.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/icu_admission-last10-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/icu_admission-last10-few

In [10]:
logging.info("Create Unplanned Admit Dataset")
unplanned_admit = builder.create_dataset(dataset_type="unplanned_admit")

INFO:root:Create Unplanned Admit Dataset
INFO:BuilderLogger:Starting Num Cases: 125881
INFO:BuilderLogger:Num Cases (filter by ProcIDs where we have PreAnes & Last10 Notes): 46472
INFO:BuilderLogger:Remove Organ Donor Cases. Num Cases: 46460
INFO:BuilderLogger:Unplanned Admit Num Cases: 595
INFO:BuilderLogger:Not Unplanned Admit Num Cases: 45865


Select up to 1 cases per Patient (Dispatch):   0%|          | 0/593 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/593 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Dispatch):   0%|          | 0/41715 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/41715 [00:00<?, ?it/s]

INFO:BuilderLogger:1 Case Per Patient. Unplanned Admit Num Cases: 593
INFO:BuilderLogger:1 Case Per Patient. Not Unplanned Admit Num Cases: 41715
INFO:BuilderLogger:1 Case Per Patient. Not Unplanned Admit Num Cases: 41715
INFO:BuilderLogger:After Downsample. Num Cases: 1186
INFO:BuilderLogger:Preanes Inference Split: 949
INFO:BuilderLogger:Preanes Fewshot Split: 237
INFO:BuilderLogger:Last10 Inference Split: 949
INFO:BuilderLogger:Last10 Fewshot Split: 237
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/unplanned_admit-preanes-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/unplanned_admit-preanes-fewshot.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/unplanned_admit-last10-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/pro

In [11]:
logging.info("Create Hospital Mortality Dataset")
hospital_mortality = builder.create_dataset(dataset_type="hospital_mortality")

INFO:root:Create Hospital Mortality Dataset
INFO:BuilderLogger:Starting Num Cases: 125881
INFO:BuilderLogger:Num Cases (filter by ProcIDs where we have PreAnes & Last10 Notes): 46472
INFO:BuilderLogger:Remove Organ Donor Cases. Num Cases: 46460
INFO:BuilderLogger:Hospital Mortality Num Cases: 410
INFO:BuilderLogger:Not Hospital Mortality Num Cases: 46050


Select up to 1 cases per Patient (Dispatch):   0%|          | 0/360 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/360 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Dispatch):   0%|          | 0/41948 [00:00<?, ?it/s]

Select up to 1 cases per Patient (Completed):   0%|          | 0/41948 [00:00<?, ?it/s]

INFO:BuilderLogger:1 Case Per Patient. Hospital Mortality Num Cases: 360
INFO:BuilderLogger:1 Case Per Patient. Not Hospital Mortality Num Cases: 41948
INFO:BuilderLogger:1 Case Per Patient. Not Hospital Mortality Num Cases: 41948
INFO:BuilderLogger:After Downsample. Num Cases: 720
INFO:BuilderLogger:Preanes Inference Split: 576
INFO:BuilderLogger:Preanes Fewshot Split: 144
INFO:BuilderLogger:Last10 Inference Split: 576
INFO:BuilderLogger:Last10 Fewshot Split: 144
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_mortality-preanes-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_mortality-preanes-fewshot.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-asa-los/data/v7/processed/dataset4/hospital_mortality-last10-inference.feather
INFO:FileLogger:Saved: /home/azureuser/cloudfiles/code/Users/chungph/llm-as